# Project Overview

Sistem rekomendasi saat ini menjadi salah satu teknologi yang sangat penting dalam dunia digital, khususnya pada platform streaming film. Banyak pengguna merasa kesulitan menemukan film yang sesuai dengan preferensi mereka karena banyaknya pilihan yang tersedia. Oleh karena itu, sistem rekomendasi dibutuhkan untuk membantu pengguna menemukan konten yang relevan dan sesuai dengan minatnya.

Proyek ini bertujuan untuk membangun sistem rekomendasi film dengan memanfaatkan dataset MovieLens 100k. Dataset ini berisi data rating dari pengguna terhadap sejumlah film, serta metadata film seperti judul dan genre. Dengan memanfaatkan data tersebut, sistem rekomendasi dapat memberikan saran film kepada pengguna berdasarkan histori rating atau kemiripan konten film.

Model sistem rekomendasi ini akan dibangun menggunakan dua pendekatan utama, yaitu:
- **Content-Based Filtering**
- **Collaborative Filtering**

Tujuan utama dari proyek ini adalah menghasilkan sistem yang mampu memberikan rekomendasi **Top-N Movie** yang relevan kepada pengguna tertentu.


# Business Understanding

## Problem Statements

Pengguna sering kesulitan dalam memilih film karena terlalu banyak pilihan. Tanpa sistem rekomendasi yang efektif, mereka bisa kehilangan minat untuk menonton.

## Goals

- Membangun sistem rekomendasi film menggunakan data rating dan metadata
- Menghasilkan Top-N rekomendasi
- Menerapkan pendekatan Content-Based dan Collaborative Filtering

## Solution Approach

1. **Content-Based Filtering**: Berdasarkan genre dan konten film
2. **Collaborative Filtering**: Berdasarkan pola rating antar pengguna


# Data Understanding

Dataset MovieLens 100k terdiri dari:
- `u.data`: data rating (user_id, item_id, rating, timestamp)
- `u.item`: metadata film (title, genre, release_date, dll)

Langkah pertama adalah memuat dan menggabungkan data.


In [ ]:
import pandas as pd

ratings = pd.read_csv("ml-100k/u.data", sep="\t", names=["user_id", "item_id", "rating", "timestamp"])
movie_cols = ["movie_id", "title", "release_date", "video_release_date", "imdb_url"] + [f"genre_{i}" for i in range(19)]
movies = pd.read_csv("ml-100k/u.item", sep="|", encoding="latin-1", header=None, names=movie_cols)

df = pd.merge(ratings, movies, left_on="item_id", right_on="movie_id")
df.head()

# Data Preparation

Langkah:
- Hapus kolom tidak penting
- Siapkan data untuk content-based (genre matrix)
- Siapkan user-item matrix untuk collaborative filtering


In [ ]:
genre_cols = [col for col in df.columns if 'genre_' in col]

# Untuk Content-Based
content_df = df.drop_duplicates(subset='movie_id')[['movie_id', 'title'] + genre_cols]

# Untuk Collaborative Filtering
user_item_matrix = df.pivot_table(index='user_id', columns='movie_id', values='rating')
user_item_matrix.head()

# Modeling and Result - Content-Based Filtering

Menggunakan cosine similarity antar fitur genre.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

genre_matrix = content_df[genre_cols].values
cosine_sim = cosine_similarity(genre_matrix, genre_matrix)
sim_df = pd.DataFrame(cosine_sim, index=content_df['title'], columns=content_df['title'])

def recommend_similar_movies(film_title, top_n=5):
    if film_title not in sim_df.columns:
        return f"Film '{film_title}' tidak ditemukan."
    sim_scores = sim_df[film_title].sort_values(ascending=False)
    return sim_scores.iloc[1:top_n+1]

recommend_similar_movies("Star Wars (1977)", top_n=5)

# Modeling and Result - Collaborative Filtering

Menggunakan Surprise library dan algoritma KNNBasic.


In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
algo = KNNBasic(sim_options={"name": "cosine", "user_based": False})
algo.fit(trainset)

predictions = algo.test(testset)
accuracy.rmse(predictions)

In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n(predictions, n=5)
for movie_id, est_rating in top_n[1]:
    title = movies[movies['movie_id'] == movie_id]['title'].values[0]
    print(f"{title} (perkiraan rating: {est_rating:.2f})")

# Evaluation

## Content-Based Filtering
Memberikan film berdasarkan genre yang mirip. Cocok untuk sistem sederhana atau saat data pengguna terbatas.

## Collaborative Filtering
Menggunakan rating pengguna lain untuk menghasilkan rekomendasi. RMSE digunakan untuk mengukur performa model prediktif.

RMSE rendah menandakan akurasi model lebih baik.

**Kesimpulan**: Collaborative Filtering lebih unggul dari sisi evaluasi numerik.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Evaluasi Tambahan: Precision@K

Metrik ini digunakan untuk mengevaluasi relevansi rekomendasi yang diberikan sistem. Precision@K menghitung proporsi dari K rekomendasi teratas yang benar-benar relevan (rating ≥ 4).


In [ ]:

def precision_at_k(top_n, threshold=4.0):
    precisions = []
    for uid, user_ratings in top_n.items():
        n_rel = sum((est >= threshold) for (_, est) in user_ratings)
        precisions.append(n_rel / len(user_ratings))
    return sum(precisions) / len(precisions)

print("Precision@5:", precision_at_k(top_n, threshold=4.0))


# Final Conclusion

Berdasarkan eksperimen yang dilakukan:

- **Content-Based Filtering** memberikan hasil rekomendasi yang baik berdasarkan genre film. Namun, model ini tidak memperhitungkan preferensi pengguna lain dan cenderung statis.
- **Collaborative Filtering (KNNBasic)** menghasilkan rekomendasi yang lebih personal dengan mempertimbangkan pola rating dari pengguna lain. Model ini juga dapat dievaluasi secara numerik dengan **RMSE** dan **Precision@K** yang menunjukkan performa cukup baik.

Secara keseluruhan, pendekatan **Collaborative Filtering** lebih unggul untuk membangun sistem rekomendasi film yang akurat dan adaptif terhadap preferensi pengguna.
